# Agilepy science API tutorial - VELA Region analysis

* The purpose of this notebook is to show how to use the Agilepy scientific API. 
* You DON'T need all the AGILE data archive to run this notebook!


## The complete API documentation can be found [at this link](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html).

## Importing the library

In [1]:
from agilepy.api.AGAnalysis import AGAnalysis

In [2]:
# Interactive plot
%matplotlib widget

## Creating a configuration file

In [3]:
confFilePath = "./agilepy_conf.yaml"

In [4]:
AGAnalysis.getConfiguration(
    evtfile="$AGILE/agilepy-test-data/test_dataset_6.0/EVT/EVT.index",
    logfile="$AGILE/agilepy-test-data/test_dataset_6.0/LOG/LOG.index",
    confFilePath = confFilePath,
    userName = "user-xxx",
    sourceName = "vela-xxx",
    tmin = 433857532,
    tmax = 435153532,
    timetype = "TT",
    glon = 263.55,
    glat = -2.78,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 1
)

## Initialization of the class

In [5]:
ag = AGAnalysis("./agilepy_conf.yaml")

2021-01-02 15:38:03,379 [INFO    ] [AgilepyLogger] File and Console loggers are active. Log file: /Users/bulgarelli/agilepy_analysis/vela-xxx_user-xxx_20210102-153803/logs/analysis_log.log


OSError: [Errno 22] Invalid argument

## API: configuration file 

In [ ]:
#print all options of the configuration file
ag.printOptions()

In [ ]:
#print a single section of the yaml configuration file
ag.printOptions("maps")

In [ ]:
#print a single option
ag.getOption("energybins")

In [ ]:
#set some options
ag.setOptions(energybins=[[100,300], [300, 1000]], mapsize=50, binsize=0.4)

In [ ]:
ag.printOptions("maps")

## API: interacting with the Sources Library

### Loading the AGILE's second catalog of gamma-ray sources

In [ ]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 5))

In [ ]:
type(sources[0])

In [ ]:
for s in sources:
    print(s)

### Alternative solution: loading the sources from a custom AGILE .txt file
The sources that are already present in the Sources Library will not be loaded twice.

In [ ]:
sources_hypotesis = """
15.7017e-08 80.3286 1.12047 2.16619 0 2 2AGLJ2032+4135 0 0 0 0 0.5 5 20 10000 0 100
16.9737e-08 79.3 0.55 2.1 3 2 CYGX3 0 0 0 0 0.5 5 20 10000 0 100
119.303e-08 78.2375 2.12298 1.75823 0 2 2AGLJ2021+4029 0 1 3307.63 0 0.5 5 20 10000 0 100
"""
with open("./sources_hyp.txt", "w") as sf:
    sf.write(sources_hypotesis)

In [ ]:
sources = ag.loadSourcesFromFile("./sources_hyp.txt", rangeDist = (0, 5))

### Selecting the sources
The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

In [ ]:
selectedSources = ag.selectSources('flux > 0')

In [ ]:
len(selectedSources)

In [ ]:
selectedSources = ag.selectSources('name == "2AGLJ2021+4029"')

In [ ]:
len(selectedSources)
for s in selectedSources:
    print(s)

In [ ]:
selectedSources = ag.selectSources("flux > 0 AND dist <= 2")

In [ ]:
len(selectedSources)
for s in selectedSources:
    print(s)

### Free/Fix source parameters
You can fix or free the following parametes:

* flux
* index
* index1
* index2
* cutoffEnergy
* pivotEnergy
* curvature
* index2

In [ ]:
_sources = ag.freeSources('name == "2AGLJ2021+4029"', "flux", True, show=True)

In [ ]:
for s in _sources:
    print(s)

In [ ]:
_sources = ag.freeSources('name == "2AGLJ2021+4029"', "index", True)

In [ ]:
for s in _sources:
    print(s)

In [ ]:
_sources = ag.freeSources("flux > 0 AND dist <= 2", "flux", True)

In [ ]:
len(_sources)

In [ ]:
for s in _sources:
    print(s)

Resetting the changes:

In [ ]:
_sources = ag.freeSources('name == "2AGLJ2021+4029"', "flux", False)

In [ ]:
ag.freeSources('name == "2AGLJ2021+4029"', "index", False)

In [ ]:
ag.freeSources("flux > 0 AND dist <= 2", "flux", False)

### Deleting sources

In [ ]:
deleted = ag.deleteSources('name == "2AGLJ2029+4403"').pop()
print(deleted)

In [ ]:
len(ag.getSources())

### Adding a source

Passing a dictionary:

In [ ]:
newSourceDict = {
    "glon" : 81.9171,
    "glat": 3.0854,
    "spectrumType" : "PowerLaw",
    "flux": 9.97261e-08,
    "index": 1.92501
}

if ag.addSource("2AGLJ2029+4403", newSourceDict):
    print("Source loaded")

In [ ]:
len(ag.getSources())

Passing a Source object:

In [ ]:
type(deleted)

In [ ]:
if ag.addSource("2AGLJ2029+4403", deleted):
    print("Source loaded")
else:
    print("Source is already present in the SourcesLibrary")

In [ ]:
deletedSources = ag.deleteSources('name == "2AGLJ2029+4403"')

In [ ]:
len(ag.getSources())

## API: science

### Generating maps

In [ ]:
ag.printOptions("maps")

In [ ]:
maplistfile = ag.generateMaps()

### Displaying maps

#### Single figure - multiple subplots

In [ ]:
ag.displayCtsSkyMaps(smooth=2, regFilePath="$AGILE/catalogs/3EG_1.reg", catalogRegions="2AGL", catalogRegionsColor="yellow")

#### Multiple figures - one subplot per figure

In [ ]:
ag.displayExpSkyMaps(singleMode = False)

In [ ]:
ag.displayGasSkyMaps()

### Saving maps

In [ ]:
ag.displayCtsSkyMaps(smooth=3, saveImage=True)

### Estimation of background coefficients
You can use the [calcBkg()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.calcBkg) method to estimate the background coefficients.

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

In [ ]:
isoBkg, galBkg, maplistfile = ag.calcBkg("2AGLJ2021+4029", galcoeff = [0.7, 0.7], pastTimeWindow = 0)

In [ ]:
# calcBkg with pastTimeWindow = 14
# ag.setOptions(galcoeff=[0.469176, 0.699403, 0.199108, 0.533379], isocoeff=[4.27314, 0.976644, 8.12386, 1.24627])

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

### Maximum likelyhood analysis

In [ ]:
# This method returns the selected sources affected by the change
sources = ag.freeSources('name == "2AGLJ2021+4029"', "flux", True)

In [ ]:
sourceFiles = ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")
len(sources)

In [ ]:
ag.freeSources('name == "2AGLJ2021+4029"', "pos", True, show=True)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")
len(sources)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")
len(sources)

## Aperture Photometry

In [ ]:
ap_file, _ = ag.aperturePhotometry()

In [ ]:
! head -n 5 $ap_file

In [ ]:
ag.displayLightCurve("ap")

## Light curve

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ2021+4029", binsize=86400)

In [ ]:
! cat $lightCurveData

In [ ]:
ag.displayLightCurve("mle")

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()